In [27]:
import numpy as np

import tensorflow as tf

from tensorflow.keras.datasets import mnist

# layers ：层，作用方便的构建层
# Model模型的意思，算法，数学公式，DNN，深度神经网络，3层隐含层 128,256,512
from tensorflow.keras import Model,layers

In [28]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()
# 归一化
X_train = (X_train/255.0).astype(np.float32)
X_test = (X_test/255.0).astype(np.float32)

# reshape,二维的数据，矩阵运算了
X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)

# one-hot 概率表示类别
y_train = tf.one_hot(y_train,depth=10)
y_test = tf.one_hot(y_test,depth=10)

# prefetch fetch取出来的意思，预先取出来，更快，加快计算速度
# 训练数据
data_train = tf.data.Dataset.from_tensor_slices((X_train,y_train)).repeat(50).shuffle(1000).batch(512).prefetch(1)
# 测试数据，验证模型好坏！每训练50次，取出一批测试数据，进行测试准确率
data_test = tf.data.Dataset.from_tensor_slices((X_test,y_test)).repeat(10).shuffle(1000).batch(512).prefetch(1)

### 构建模型，损失，准确率，优化算法

In [29]:
class Dnn(Model):
    def __init__(self):
        super(Dnn,self).__init__()#调用父类方法
#         没有写，变量，layers自动帮助，写变量Variable，对应着神经元数量
        self.h1 = layers.Dense(256,activation='relu')# 矩阵运算，Dense相当于矩阵运算
        self.h2 = layers.Dense(256,activation='relu')
        self.h3 = layers.Dense(512,activation='relu')
        self.out = layers.Dense(10,activation=tf.nn.softmax)# softmax相当于概率！
    def call(self,x,is_training = False):# 可以帮助我们区分，训练和测试情况
        x = self.h1(x)
        x = self.h2(x)
        x = self.h3(x)
        y_pred = self.out(x)
        return y_pred
dnn = Dnn()
def cross_entropy(y_true,y_pred):
    y_pred = tf.clip_by_value(y_pred,1e-9,1.0)
    loss = tf.reduce_mean(tf.reduce_sum(-tf.multiply(y_true,tf.math.log(y_pred)),axis = 1))#y_pred是在分母的位置
    return loss
# 声明准确率
def accuracy(y_true,y_pred):#此时y_true,y_pred都是tensor
#     这种写法，和上一个案例，不同，意思完全一样！
    y_true = tf.argmax(y_true,axis = -1)
    y_pred = tf.argmax(y_pred,axis = -1)
#     numpy 支持 == ，Tensorflow 不支持==，必须使用equal 比较的是值，==比较的是内存地址
    acc = tf.reduce_mean(tf.cast(tf.equal(y_true,y_pred),tf.float16)).numpy()
    return acc
sgd = tf.optimizers.SGD(0.1)

### 定义优化方法

In [30]:
def run_optimizer(X_train,y_train):
    with tf.GradientTape() as g:
        y_pred = dnn(X_train)
        loss = cross_entropy(y_train,y_pred)
    gradients = g.gradient(loss,dnn.trainable_variables)#获取dnn模型的为我们声明的变量
    sgd.apply_gradients(zip(gradients,dnn.trainable_variables))

### for循环，批量的训练

In [31]:
for i,(X_train,y_train) in enumerate(data_train.take(1000),1):
    run_optimizer(X_train,y_train)
    if i%10 == 0:
        for (X_test,y_test) in data_test.take(1):
            y_pred = dnn(X_test)
            acc = accuracy(y_test,y_pred)
            print('执行次数：%d。准确率：%0.4f'%(i,acc))

执行次数：10。准确率：0.6523
执行次数：20。准确率：0.7285
执行次数：30。准确率：0.7734
执行次数：40。准确率：0.8047
执行次数：50。准确率：0.8047
执行次数：60。准确率：0.8672
执行次数：70。准确率：0.8730
执行次数：80。准确率：0.8672
执行次数：90。准确率：0.8750
执行次数：100。准确率：0.8711
执行次数：110。准确率：0.8848
执行次数：120。准确率：0.8633
执行次数：130。准确率：0.8945
执行次数：140。准确率：0.8887
执行次数：150。准确率：0.8887
执行次数：160。准确率：0.8770
执行次数：170。准确率：0.8867
执行次数：180。准确率：0.9160
执行次数：190。准确率：0.9043
执行次数：200。准确率：0.8984
执行次数：210。准确率：0.8926
执行次数：220。准确率：0.8965
执行次数：230。准确率：0.8984
执行次数：240。准确率：0.9023
执行次数：250。准确率：0.8906
执行次数：260。准确率：0.9199
执行次数：270。准确率：0.9141
执行次数：280。准确率：0.9102
执行次数：290。准确率：0.9062
执行次数：300。准确率：0.9121
执行次数：310。准确率：0.9180
执行次数：320。准确率：0.9141
执行次数：330。准确率：0.8867
执行次数：340。准确率：0.9062
执行次数：350。准确率：0.8926
执行次数：360。准确率：0.9219
执行次数：370。准确率：0.9023
执行次数：380。准确率：0.9512
执行次数：390。准确率：0.9160
执行次数：400。准确率：0.8867
执行次数：410。准确率：0.9160
执行次数：420。准确率：0.9336
执行次数：430。准确率：0.9238
执行次数：440。准确率：0.9199
执行次数：450。准确率：0.9238
执行次数：460。准确率：0.9297
执行次数：470。准确率：0.9258
执行次数：480。准确率：0.9355
执行次数：490。准确率：0.9121
执行次数：500。准确率：0.8984
执行次数：510。